# Attention Head
* so in transformer we do use multiplicative attention ,there are two type of attention one is additive and other is scale dot attention
* so in research paper  "attention all u need" they hav incorported multiplicative attention
* i assume it just involves addition rather than the multiplication to calculate the score

dimension of data (directly proportional to) variance 

hence the scailing down is down by (dimension of model)**(1/2)


In [ ]:
import torch as nn

In [ ]:
#it has been modified to process the 3d data where the data across the z index correspond to each head

def attention_head(queries, keys, values, dropout=None, mask=None):

  # Calculate the attention scores
  # Scores will have shape (batch_size * sequence_length, batch_size * sequence_length)
  d_k = keys.size(-1)
  scores = nn.matmul(queries, keys.transpose(-2, -1))
  scores = scores / d_k**0.5

#now this look ahead mask will be used in the decoder
  if mask is not None:
# mask should be broadcastable to the shape of the scores tensor
# for 3d input, the mask should have shape (num_heads,batch_size * sequence_length, batch_size * sequence_length)
      scores = scores.masked_fill(mask == 0, -1e9)

  # row mean -1 and -2 means col
  scores = scores.softmax(dim=-1) # apply softmax across the row dimension (keys)


  # Apply dropout
  if dropout is not None:
      scores = dropout(scores)

  # Apply scores to values
  weighted_sum = nn.matmul(scores, values)

  return weighted_sum

### we will test our attention head with custom args

In [ ]:
import torch

In [ ]:
# Define input dimensions
batch_size = 2
sequence_length = 3
embedding_dim = 4
num_heads = 2
head_dim = embedding_dim // num_heads

# Create sample input tensors with compatible shapes
# Embeddings are now 2D: (batch_size * sequence_length, embedding_dim)
sample_embeddings = torch.randn(batch_size * sequence_length, embedding_dim)

# Create dummy linear layers that output the expected shape for attention_head
# The linear layers now expect an input shape of (batch_size * sequence_length, embedding_dim)
# and output a shape of (batch_size * sequence_length, head_dim)
dummy_key_linear = torch.nn.Linear(embedding_dim, head_dim)
dummy_query_linear = torch.nn.Linear(embedding_dim, head_dim)
dummy_value_linear = torch.nn.Linear(embedding_dim, head_dim)

# Create a dummy dropout layer
dummy_dropout = torch.nn.Dropout(0.1)

# Create a dummy mask
# The mask should have shape (batch_size * sequence_length, batch_size * sequence_length) for 2D input
dummy_mask = torch.ones(batch_size * sequence_length, batch_size * sequence_length) # Assuming no masking for simplicity

# Pass the sample_embeddings through the dummy linear layers to get the keys, queries, and values tensors
# These tensors should have shape (batch_size * sequence_length, head_dim)
sample_keys = dummy_key_linear(sample_embeddings)
sample_queries = dummy_query_linear(sample_embeddings)
sample_values = dummy_value_linear(sample_embeddings)

print("Sample Keys Shape:", sample_keys.shape)
print("Sample Queries Shape:", sample_queries.shape)
print("Sample Values Shape:", sample_values.shape)

# Test the attention_head function with the compatible inputs

  # attention_head now expects 2D inputs for keys, queries, and values
attention_output = attention_head(sample_queries, sample_keys, sample_values, dummy_dropout, dummy_mask)
print("Attention head output shape:", attention_output.shape)


# testing for the 3d inputs

In [ ]:
# Define input dimensions
batch_size = 2
sequence_length = 3
embedding_dim = 4
num_heads = 2
head_dim = embedding_dim // num_heads

# Create sample input tensors with compatible shapes
# Embeddings are now 2D: (batch_size * sequence_length, embedding_dim)
sample_embeddings = torch.randn(batch_size * sequence_length, embedding_dim)

# Create dummy linear layers that output the expected shape for attention_head
# The linear layers now expect an input shape of (batch_size * sequence_length, embedding_dim)
# and output a shape of (batch_size * sequence_length, head_dim)
dummy_key_linear = torch.nn.Linear(embedding_dim, head_dim)
dummy_query_linear = torch.nn.Linear(embedding_dim, head_dim)
dummy_value_linear = torch.nn.Linear(embedding_dim, head_dim)

# Create a dummy dropout layer
dummy_dropout = torch.nn.Dropout(0.1)

# Create a dummy mask
# The mask should have shape (batch_size * sequence_length, batch_size * sequence_length) for 2D input
dummy_mask = torch.ones(batch_size * sequence_length, batch_size * sequence_length) # Assuming no masking for simplicity

# Pass the sample_embeddings through the dummy linear layers to get the keys, queries, and values tensors
# These tensors should have shape (batch_size * sequence_length, head_dim)
sample_keys = dummy_key_linear(sample_embeddings)
sample_queries = dummy_query_linear(sample_embeddings)
sample_values = dummy_value_linear(sample_embeddings)

print("Sample Keys Shape:", sample_keys.shape)
print("Sample Queries Shape:", sample_queries.shape)
print("Sample Values Shape:", sample_values.shape)

# Test the attention_head function with the compatible inputs

  # attention_head now expects 2D inputs for keys, queries, and values
attention_output = attention_head(sample_queries, sample_keys, sample_values, dummy_dropout, dummy_mask)
print("Attention head output shape:", attention_output.shape)


# Multi Head Attention 
#### we create tunable parameters for each head one for query ,key and value 

In [ ]:
import torch.nn as nn
import torch

class MultiHeadAttention(nn.Module):
  def __init__(self,heads,d_model,dropout=0.01):
     super(MultiHeadAttention,self).__init__();
     self.heads=heads;
     self.d_model=d_model;
     self.d_key=d_model//heads;
     # Linear layers for query, key, value, and projection
     self.query_linears = clones(nn.Linear(d_model, d_model//self.heads),self.heads)
     self.key_linears =   clones(nn.Linear(d_model, d_model//self.heads),self.heads)
     self.value_linears = clones(nn.Linear(d_model, d_model//self.heads),self.heads)
     #prjection layer will be used to project the concatenated output of all heads to the end of attention in the multihead attention
     self.projection_layer=nn.Linear(d_model,self.d_key)
     self.dropout=nn.Dropout(dropout);

  def forward(self,input_embedding):
    batch_size = input_embedding.size(0)
    result=[]
    for i in range(0,self.heads):
        out=attention_head(input_embedding, self.query_linears[i], self.key_linears[i], self.value_linears[i],self.dropout)
        result.append(out)
    final_result = torch.cat(result_list, dim=1)
    return self.projection_layer(final_result)


In [ ]:
import torch

# Define parameters for MultiHeadAttention
heads = 4
d_model = 128 # Make sure d_model is divisible by heads
dropout_prob = 0.1

# Create a dummy input embedding
batch_size = 2
sequence_length = 10
input_embedding = torch.randn(batch_size, sequence_length, d_model)

# Create an instance of MultiHeadAttention
multi_head_attention = MultiHeadAttention(heads, d_model, dropout=dropout_prob)

# Test the forward method
output = multi_head_attention(input_embedding)

print("Input embedding shape:", input_embedding.shape)
print("Output shape:", output.shape)